# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [13]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/cimsfo/fosearch.asp')

In [14]:
from selenium.webdriver.support.ui import Select
select = Select(driver.find_element_by_name('pht_status'))
select.select_by_visible_text('Cosmetologists')

In [15]:
text_input = driver.find_element_by_name('pht_lnm')
text_input.send_keys('Nguyen')

In [16]:
search_button = driver.find_element_by_name('B1')
search_button.click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [17]:
cosmetologists = driver.find_elements_by_tag_name('tr')
for cosmetologist in cosmetologists[:10]:
    print(cosmetologist.text)

Name and Location Order Basis for Order
NGUYEN, TOAN HUU
City: SAN ANTONIO
County: BEXAR
Zip Code: 78217


License #(s): 780948, 1706491, 1699123

Complaint # COS20180004289 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $500. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
NGUYEN, HANH CONG
City: EL PASO
County: EL PASO
Zip Code: 79934


License #: 737708

Complaint # COS20180006594 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $1,000. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
NGUYEN, KHIEM VAN
City: LONGVIEW
County: GREGG
Zip Code: 75604


License #: 731665

Complaint # COS20180000257 Date: 5/17/2018

Respondent is assessed an administrative penalty in the amount of $1,250. Responde

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

In [7]:
for cosmetologist in cosmetologists[:10]:
    try:
        print(cosmetologist.find_element_by_class_name('results_text').text)
    except:
        pass

NGUYEN, TOAN HUU
NGUYEN, HANH CONG
NGUYEN, KHIEM VAN
NGUYEN, DIEP THI NGOC
NGUYEN, LAN T-THUY
NGUYEN, TUAN A
NGUYEN, THAO B
NGUYEN, BETH MARIA
NGUYEN, TRUNG N


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...

In [8]:
for cosmetologist in cosmetologists[:10]:
    try:
        print(cosmetologist.find_elements_by_tag_name('td')[2].text)
    except:
        pass

Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect all wax pots.
Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution; Respondent failed to disinfect multi-use equipment, implements, and tools prior to use on each client.
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
Respondent failed to clean and disinfect all wax pots; Respondent failed to disinfect tools, implements, an

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [9]:
for cosmetologist in cosmetologists[:10]:
    try:
        print(cosmetologist.find_elements_by_class_name('results_text')[-2].text)
    except:
        print('It didn\'t work.')

It didn't work.
COS20180004289
COS20180006594
COS20180000257
COS20180004915
COS20180009255
COS20140018343
COS20180008846
COS20180000897
COS20170023893


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [18]:
rows = []
for cosmetologist in cosmetologists[1:]:
    row = {}
    
    Name = cosmetologist.find_element_by_class_name('results_text')
    row['Name'] = Name.text
    
    Violation_description = cosmetologist.find_elements_by_tag_name('td')[2]
    row['Violation_description'] = Violation_description.text
    
    Complaint_number = cosmetologist.find_elements_by_class_name('results_text')[-2]
    row['Complaint_number'] = Complaint_number.text
    
    License_numbers = cosmetologist.find_elements_by_class_name('results_text')[-3]
    row['License_numbers'] = License_numbers.text
    
    Zip = cosmetologist.find_elements_by_class_name('results_text')[-4]
    row['Zip'] = Zip.text
    
    County = cosmetologist.find_elements_by_class_name('results_text')[-5]
    row['County'] = County.text
    
    City = cosmetologist.find_elements_by_class_name('results_text')[-6]
    row['City'] = City.text
    
    rows.append(row)

### Save that to a CSV

- Tip: You'll want to use pandas here

In [19]:
import pandas as pd

df = pd.DataFrame(rows)
df.head(10)

,City,Complaint_number,County,License_numbers,Name,Violation_description,Zip
0,SAN ANTONIO,COS20180004289,BEXAR,"780948, 1706491, 1699123","NGUYEN, TOAN HUU",Respondent failed to clean and sanitize whirlp...,78217
1,EL PASO,COS20180006594,EL PASO,737708,"NGUYEN, HANH CONG",Respondent failed to clean and sanitize whirlp...,79934
2,LONGVIEW,COS20180000257,GREGG,731665,"NGUYEN, KHIEM VAN",Respondent failed to follow whirlpool foot spa...,75604
3,HOUSTON,COS20180004915,HARRIS,"1347649, 760528","NGUYEN, DIEP THI NGOC","Respondent failed to disinfect tools, implemen...",77014
4,SAN ANTONIO,COS20180009255,BEXAR,767339,"NGUYEN, LAN T-THUY","Respondent failed to clean, disinfect, and ste...",78255
5,ARLINGTON,COS20140018343,TARRANT,681274,"NGUYEN, TUAN A",Respondent failed to clean and disinfect all w...,76011
6,EULESS,COS20180008846,TARRANT,"721373, 1142884","NGUYEN, THAO B",Respondent failed to clean and sanitize whirlp...,76039
7,HOUSTON,COS20180000897,HARRIS,1470271,"NGUYEN, BETH MARIA",The Respondent's license was revoked upon Resp...,77083
8,AMARILLO,COS20170023893,POTTER,"1196244, 767015, 767014","NGUYEN, TRUNG N","Respondent failed to clean, disinfect, and ste...",79106
9,PITTSBURG,COS20180004076,CAMP,759931,"NGUYEN, NGAT THI",Respondent failed to follow whirlpool foot spa...,75686


In [20]:
df.to_csv("cosmetology_violations.csv", index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

NameError: name 'cosmetology_violations' is not defined